# # AI-Powered Food Safety Compliance System
# This notebook implements a food safety compliance system using CrewAI and Gradio.



# ## 1. Install Dependencies
# First, let's install the required packages.

In [3]:
!pip install gradio crewai langchain-groq duckduckgo-search python-dotenv

  Using cached gradio-5.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached crewai-0.76.9-py3-none-any.whl.metadata (18 kB)
  Using cached duckduckgo_search-6.3.4-py3-none-any.whl.metadata (25 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.4-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.4.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.12-py3-none-any.whl.metadata (1.9 kB)
  Using cached ruff-0.7.3-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.1-py3-none-any.whl.m

# 2. Import Libraries

In [4]:
import os
import gradio as gr
from crewai import Agent, Task, Crew, Process
from langchain.chat_models import ChatLiteLLM
from langchain.tools import DuckDuckGoSearchRun
import json
from datetime import datetime
import logging
import traceback
from litellm import completion
from google.colab import userdata

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 3. Configuration Setup
# Initialize API keys and core components


In [5]:
def initialize_environment():
    try:
        GROQ_API_KEY = os.getenv("GROQ_API_KEY")
        if not GROQ_API_KEY:
            GROQ_API_KEY = userdata.get('GROQ_API_KEY')
            os.environ["GROQ_API_KEY"] = GROQ_API_KEY

        groq_llm = ChatLiteLLM(
            model_name="groq/mixtral-8x7b-32768",
            temperature=0.3,
            max_tokens=32768,
            api_key=GROQ_API_KEY
        )
        search_tool = DuckDuckGoSearchRun()

        return groq_llm, search_tool

    except Exception as e:
        logger.error(f"Initialization error: {str(e)}")
        logger.error(traceback.format_exc())
        raise e

# 4. Agent Definitions
# Define the specialized agents for food safety monitoring


In [7]:
class FoodSafetySystem:
    def __init__(self):
        try:
            logger.info("Initializing FoodSafetySystem agents...")
            self.llm, self.search_tool = initialize_environment()

            self.haccp_monitor = Agent(
                role='HACCP Monitor',
                goal='Monitor critical control points and food safety procedures',
                backstory='Expert in HACCP principles and food safety monitoring',
                llm=self.llm,
                tools=[self.search_tool],
                verbose=True
            )

            self.quality_inspector = Agent(
                role='Quality Inspector',
                goal='Assess food quality parameters and safety standards',
                backstory='Specialist in food quality control and safety standards',
                llm=self.llm,
                tools=[self.search_tool],
                verbose=True
            )

            self.compliance_auditor = Agent(
                role='Compliance Auditor',
                goal='Audit documentation and regulatory compliance',
                backstory='Expert in food safety regulations and compliance auditing',
                llm=self.llm,
                tools=[self.search_tool],
                verbose=True
            )

            self.risk_predictor = Agent(
                role='Risk Predictor',
                goal='Predict potential food safety risks and violations',
                backstory='Specialist in predictive analytics for food safety',
                llm=self.llm,
                tools=[self.search_tool],
                verbose=True
            )

            logger.info("Successfully initialized all agents")

        except Exception as e:
            logger.error(f"Error initializing agents: {str(e)}")
            logger.error(traceback.format_exc())
            raise e
    def run_safety_assessment(self, documents: list) -> list:
        try:
            logger.info("Starting safety assessment")

            tasks = [
                Task(
                    description=f"Analyze HACCP procedures in: {documents}",
                    agent=self.haccp_monitor,
                    expected_output="A detailed analysis of HACCP procedures including critical control points, monitoring procedures, and any deviations identified."
                ),
                Task(
                    description=f"Assess quality control in: {documents}",
                    agent=self.quality_inspector,
                    expected_output="A comprehensive assessment of quality control measures including test results, product specifications, and quality metrics."
                ),
                Task(
                    description=f"Audit compliance in: {documents}",
                    agent=self.compliance_auditor,
                    expected_output="A detailed compliance audit report highlighting regulatory requirements, compliance status, and any identified gaps."
                ),
                Task(
                    description=f"Predict risks in: {documents}",
                    agent=self.risk_predictor,
                    expected_output="A risk assessment report identifying potential food safety hazards, their likelihood, and recommended preventive measures."
                )
            ]

            crew = Crew(
                agents=[self.haccp_monitor, self.quality_inspector,
                        self.compliance_auditor, self.risk_predictor],
                tasks=tasks,
                process=Process.sequential,
                verbose=True
            )

            results = crew.kickoff()
            return [str(r) if r is not None else "No results available" for r in results]

        except Exception as e:
            logger.error(f"Error in safety assessment: {str(e)}")
            logger.error(traceback.format_exc())
            raise e




#  5. Processing Function
# Define the main processing function that handles the workflow


In [8]:
def process_food_safety(
    production_docs: str,
    haccp_plan: str,
    quality_records: str,
    progress=gr.Progress()
) -> tuple:
    try:
        logger.info("Starting food safety processing")
        progress(0.1, desc="Initializing...")

        if not all([production_docs.strip(), haccp_plan.strip(), quality_records.strip()]):
            raise ValueError("All input fields must be filled out")

        documents = [
            f"Production Documentation:\n{production_docs}",
            f"HACCP Plan:\n{haccp_plan}",
            f"Quality Control Records:\n{quality_records}"
        ]

        progress(0.2, desc="Running analysis...")
        safety_system = FoodSafetySystem()
        results = safety_system.run_safety_assessment(documents)

        progress(0.8, desc="Processing results...")

        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"food_safety_report_{timestamp}.json"

        with open(filename, 'w') as f:
            json.dump({
                "haccp_monitoring": results[0],
                "quality_inspection": results[1],
                "compliance_audit": results[2],
                "risk_prediction": results[3]
            }, f, indent=2)

        progress(1.0, desc="Complete!")
        return (*results, f"Report saved as: {filename}")

    except Exception as e:
        logger.error(f"Error in process_food_safety: {str(e)}")
        logger.error(traceback.format_exc())
        return (f"Error: {str(e)}",) * 5

# 7. Gradio Interface
# Create the user interface using Gradio


In [9]:
def create_interface():
    with gr.Blocks(title="Food Safety Compliance System") as interface:
        gr.Markdown("# AI-Powered Food Safety Compliance System")

        with gr.Row():
            with gr.Column():
                production_docs = gr.Textbox(
                    label="Production Documentation",
                    placeholder="Enter production details...",
                    lines=8
                )
                haccp_plan = gr.Textbox(
                    label="HACCP Plan",
                    placeholder="Enter HACCP plan...",
                    lines=8
                )
                quality_records = gr.Textbox(
                    label="Quality Control Records",
                    placeholder="Enter quality records...",
                    lines=8
                )
                submit_btn = gr.Button("Run Analysis", variant="primary")

        with gr.Row():
            with gr.Column():
                haccp_output = gr.Textbox(
                    label="HACCP Monitoring Results",
                    lines=5
                )
                quality_output = gr.Textbox(
                    label="Quality Inspection Results",
                    lines=5
                )
                compliance_output = gr.Textbox(
                    label="Compliance Audit Results",
                    lines=5
                )
                risk_output = gr.Textbox(
                    label="Risk Prediction Results",
                    lines=5
                )
                file_output = gr.Textbox(
                    label="Report Status"
                )

        submit_btn.click(
            fn=process_food_safety,
            inputs=[production_docs, haccp_plan, quality_records],
            outputs=[haccp_output, quality_output, compliance_output,
                    risk_output, file_output]
        )

    return interface

# 8. Launch Application
# Initialize and launch the Gradio interface


In [ ]:
if __name__ == "__main__":
    interface = create_interface()
    interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d2972b247beb1b2ca2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Agent: HACCP Monitor
## Task: Analyze HACCP procedures in: ['Production Documentation:\nFresh Dairy Solutions Inc.,\n      productLine: "Pasteurized Whole Milk",\n      batchNumber: "ML-2024-0428-01",\n      productionDate: "2024-04-28",\n      productionDetails: `\n        - Production Start Time: 06:00\n        - Batch Size: 5000 liters\n        - Pasteurization Temperature: 72.5°C\n        - Holding Time: 15 seconds\n        - Cooling Temperature: 4°C\n        - Equipment: Pasteurizer Unit #2\n        - Operator: J. Smith\n        - Supervisor: M. Johnson\n', 'HACCP Plan:\n      Critical Control Points (CCPs):\n      1. Pasteurization (CCP-1)\n         - Critical Limit: Min 72°C for 15 seconds\n         - Monitoring: Continuous temperature recording\n         - Corrective Action: If temperature drops below 72°C, divert flow and reprocess\n         \n      2. Cooling (CCP-2)\n         - Critical Limit: Max 4°C within 2 hours\n         - Monitoring: Temperature check every 30 minute

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1748: UserWarning: A function (process_food_safety) returned too many output values (needed: 5, returned: 6). Ignoring extra values.
    Output components:
        [textbox, textbox, textbox, textbox, textbox]
    Output values returned:
        ["('raw', 'Based on the provided Compliance Audit Report and the search results, there are deviations in cooling temperature for both the production documentation and HACCP plan, as well as in the finished product temperature in the quality control records. Although the search results do not provide specific information about risks associated with slightly elevated cooling temperature in pasteurized milk, they emphasize the importance of maintaining proper temperature for milk safety and quality. Therefore, these deviations should be taken seriously, and corrective action is required to adjust the cooling system and evaluate the product for safety.')", "('pydantic', None)", "('json_dict',